# **Setup**

**Reference**
* https://www.youtube.com/watch?v=A663NOHPRHE&list=PLB9cczsBXUdnd0pg4ybBmzbNgU9Sx4ED6&index=2

In [34]:
import numpy as np 
import pandas as pd 
from scipy.stats import norm
import yfinance as yf
from datetime import datetime, timedelta 

# **Asian Option Price Prediction**

* Using Monte Carlo
* Asian Option Pricing: Price Over Certain Duration
* European Option Pricing: Price At Maturity 

In [3]:
SO = 100        # Initial Stock Price
K = 105         # Strike Price
T = 1           # Time To Maturity
r = 0.05        # Short Rate
sigma = 0.2     # Volatility
M = 50          # Time Steps
dt = T/M        # Length Of Time Interval
I = 250000      # No. Of Simulations

In [13]:
np.random.seed(0)

# Simulate I Random Paths With M Time Steps
S = np.zeros(shape=(M+1,I))
print(S.shape)

S[0] = SO

(51, 250000)


In [32]:
for t in range(1,M+1):
    S[t] = S[t-1] * np.exp((r-0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * np.random.standard_normal(size=I))

# Options Payoff
hT = np.maximum(np.mean(S[-M:],axis=0) - K,0)

# Options Price
CO = np.exp(-r * T) * np.sum(hT)/I
print(f'Value Of Asian Option: {CO}')

Value Of Asian Option: 3.604232489398991


# **European Option Pricing**
* Using Monte Carlo Simulation

## **Get Stock Price Data**

In [40]:
stock_name = 'AAPL'
end_date = datetime.today()
start_date = end_date - timedelta(days=365)
print('Start Date',start_date)
print('End Date',end_date)
print('\n')

AAPL_stock_data = yf.download(stock_name,start=start_date,end=end_date)
AAPL_stock_data.tail(10)

Start Date 2023-01-04 13:50:44.347723
End Date 2024-01-04 13:50:44.347723


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-19,196.160004,196.949997,195.889999,196.940002,196.940002,40714100
2023-12-20,196.899994,197.679993,194.830002,194.830002,194.830002,52242800
2023-12-21,196.100006,197.080002,193.500000,194.679993,194.679993,46482500
2023-12-22,195.179993,195.410004,192.970001,193.600006,193.600006,37122800
2023-12-26,193.610001,193.889999,192.830002,193.050003,193.050003,28919300
2023-12-27,192.490005,193.500000,191.089996,193.149994,193.149994,48087700
2023-12-28,194.139999,194.660004,193.169998,193.580002,193.580002,34049900
2023-12-29,193.899994,194.399994,191.729996,192.529999,192.529999,42628800
2024-01-02,187.149994,188.440002,183.889999,185.639999,185.639999,82488700


## **Calculate Daily Returns**

In [43]:
# Compute Daily Returns
AAPL_Daily_Returns = AAPL_stock_data['Adj Close'].pct_change(1)
print(AAPL_Daily_Returns[:5])

Date
2023-01-04         NaN
2023-01-05   -0.010605
2023-01-06    0.036794
2023-01-09    0.004089
2023-01-10    0.004457
Name: Adj Close, dtype: float64


## **Calculate Stock Volatility**

In [44]:
# Compute Stock Volatility
AAPL_Stock_Volatility = AAPL_Daily_Returns.std()
print(f'Daily {stock_name} Volatility: {AAPL_Stock_Volatility}')

Daily AAPL Volatility: 0.012769754252477283


## **Calculate European Options Price**

In [72]:
# European Options Price
AAPL_Stock_Last_Price = AAPL_stock_data.loc[:,'Adj Close'].iloc[-1]
Strike_Price = AAPL_Stock_Last_Price

In [87]:
# Define Function
def European_Option_Price(S0,Strike_Price,r,v,T,nSim,flag):
    z = np.random.standard_normal(nSim)
    S_Forward = S0 * np.exp((r-0.5*v**2)*T + v*np.sqrt(T)*z)

    if flag == 'call':
        payoff = np.max(S_Forward - Strike_Price,0)
    elif flag == 'put':
        payoff = np.max(Strike_Price - S_Forward,0)
    else:
        print('Invalid Flag!')
        return
    return np.exp(-r*T) * np.sum(payoff)/nSim    

In [86]:
nSim = 100000
Risk_Free_Rate = 0.04
Maturity = 1.0 # Year

# Call Price
AAPL_Call_Option_Price = European_Option_Price(S0=AAPL_Stock_Last_Price,Strike_Price=Strike_Price,r=Risk_Free_Rate,v=AAPL_Stock_Volatility,
                      T=Maturity,nSim=nSim,flag='call')
print(AAPL_Call_Option_Price)

# Put Price
AAPL_Put_Option_Price = European_Option_Price(S0=AAPL_Stock_Last_Price,Strike_Price=Strike_Price,r=Risk_Free_Rate,v=AAPL_Stock_Volatility,
                      T=Maturity,nSim=nSim,flag='put')
print(AAPL_Put_Option_Price)

0.00017106719061666382
2.0640972393899304e-05
